In [1]:
# Chattea Intent Classifier - CNN + Sentence Transformers
# Complete Working Pipeline in Notebook Format
# Ready to run cell by cell!

"""
JUPYTER NOTEBOOK STRUCTURE:
Run each cell in order (Shift+Enter)

Required Files:
- chattea_dataset.csv (text, intent columns)
- responses.json (your bilingual responses)

Installation:
!pip install torch sentence-transformers scikit-learn pandas difflib
"""

# ============================================================================
# CELL 1: IMPORTS AND SETUP
# ============================================================================

import json
import pandas as pd
import re
import torch
import torch.nn as nn
import torch.nn.functional as F
from sentence_transformers import SentenceTransformer, util
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from difflib import get_close_matches
import warnings
warnings.filterwarnings('ignore')

print("=" * 80)
print("CHATTEA INTENT CLASSIFIER - CNN + SENTENCE TRANSFORMERS")
print("=" * 80)
print(f"PyTorch Version: {torch.__version__}")
print("=" * 80)

CHATTEA INTENT CLASSIFIER - CNN + SENTENCE TRANSFORMERS
PyTorch Version: 2.9.1+cu130


In [2]:

# ============================================================================
# CELL 2: DEVICE CONFIGURATION
# ============================================================================

# Set device (GPU if available, otherwise CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🖥️  Using device: {device}")

if torch.cuda.is_available():
    print(f"🎮 GPU: {torch.cuda.get_device_name(0)}")
    print(f"💾 GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("⚠️  No GPU detected - training will use CPU (slower)")


🖥️  Using device: cuda
🎮 GPU: NVIDIA GeForce RTX 4060 Laptop GPU
💾 GPU Memory: 8.59 GB


In [3]:
# ============================================================================
# CELL 3: LOAD DATA
# ============================================================================

print("\n" + "=" * 80)
print("📂 LOADING DATA")
print("=" * 80)

# Load training dataset
df = pd.read_csv("chattea_dataset.csv")
print(f"✓ Loaded dataset: {len(df)} samples")
print(f"✓ Columns: {list(df.columns)}")
print(f"✓ Unique intents: {df['intent'].nunique()}")

# Show sample data
print("\n📊 Sample data:")
print(df.head(10))

# Intent distribution
print("\n📈 Intent distribution:")
intent_counts = df['intent'].value_counts()
print(intent_counts.head(15))

# Load responses
with open("responses.json", "r", encoding="utf-8") as f:
    RESPONSES = json.load(f)
print(f"\n✓ Loaded responses for {len(RESPONSES)} intents")



📂 LOADING DATA
✓ Loaded dataset: 657 samples
✓ Columns: ['text', 'intent']
✓ Unique intents: 33

📊 Sample data:
           text    intent
0         hello  greeting
1            hi  greeting
2     hey there  greeting
3  good morning  greeting
4  good evening  greeting
5      hi there  greeting
6           hey  greeting
7     greetings  greeting
8     what's up  greeting
9     hello bot  greeting

📈 Intent distribution:
intent
greeting            20
gratitude           20
help                20
definition          20
cancel              20
instance_manage     20
instance_create     20
instance_connect    20
instance_status     20
message_status      20
message_send        20
message_blast       20
message_schedule    20
contacts            20
chat_view           20
Name: count, dtype: int64

✓ Loaded responses for 32 intents


In [4]:

# ============================================================================
# CELL 4: BUILD VOCABULARY FOR FUZZY MATCHING
# ============================================================================

print("\n" + "=" * 80)
print("📚 BUILDING VOCABULARY FOR FUZZY MATCHING")
print("=" * 80)

# Extract all unique words from training data for typo correction
all_words = set()
for text in df['text'].str.lower():
    all_words.update(re.findall(r'\w+', text))

VOCAB = all_words
print(f"✓ Vocabulary size: {len(VOCAB)} unique words")
print(f"✓ Sample words: {list(VOCAB)[:20]}")

def fuzzy_correct(text: str, cutoff: float = 0.8) -> str:
    """
    Typo correction using difflib (Fuzzy String Matching)
    
    Algorithm: Levenshtein Distance
    - Finds closest matching words from vocabulary
    - Corrects typos while preserving sentence structure
    
    Example: "blst mesage" → "blast message"
    """
    words = re.findall(r'\w+', text.lower())
    corrected = []
    
    for word in words:
        # Find closest match in vocabulary
        matches = get_close_matches(word, VOCAB, n=1, cutoff=cutoff)
        corrected.append(matches[0] if matches else word)
    
    # Reconstruct sentence preserving original punctuation
    result = text
    for orig, corr in zip(words, corrected):
        if orig != corr:
            result = re.sub(rf'\b{orig}\b', corr, result, count=1, flags=re.IGNORECASE)
    
    return result

# Test fuzzy correction
print("\n🧪 Testing Fuzzy Correction:")
test_cases = [
    "blst mesage",
    "chek number",
    "craete instance",
    "shedule mesage"
]

for test in test_cases:
    corrected = fuzzy_correct(test)
    print(f"   '{test}' → '{corrected}'")



📚 BUILDING VOCABULARY FOR FUZZY MATCHING
✓ Vocabulary size: 451 unique words
✓ Sample words: ['hey', 'endpoints', 'one', 'contacts', 'up', 'preferences', 'api', 'comparison', 'have', 'appreciated', 'device', 'activate', 'validation', 'engagement', 'request', 'client', 'keys', 'book', 'description', 'discard']

🧪 Testing Fuzzy Correction:
   'blst mesage' → 'blast message'
   'chek number' → 'check number'
   'craete instance' → 'create instance'
   'shedule mesage' → 'schedule message'


In [5]:

# ============================================================================
# CELL 5: LABEL ENCODING
# ============================================================================

print("\n" + "=" * 80)
print("🏷️  ENCODING LABELS")
print("=" * 80)

# Encode intent labels to numeric values
le = LabelEncoder()
df['label'] = le.fit_transform(df['intent'])

num_classes = len(le.classes_)
intent_map = dict(enumerate(le.classes_))

print(f"✓ Number of classes: {num_classes}")
print(f"\n📋 Intent mapping (first 10):")
for idx, intent in list(intent_map.items())[:10]:
    print(f"   {idx}: {intent}")


🏷️  ENCODING LABELS
✓ Number of classes: 33

📋 Intent mapping (first 10):
   0: account
   1: analytics
   2: api
   3: cancel
   4: chat_view
   5: contacts
   6: definition
   7: files
   8: goodbye
   9: gratitude


In [6]:

# ============================================================================
# CELL 6: SENTENCE EMBEDDINGS (WORD2VEC ALTERNATIVE)
# ============================================================================

print("\n" + "=" * 80)
print("🧠 GENERATING SENTENCE EMBEDDINGS")
print("=" * 80)
print("Using: Sentence Transformers (all-MiniLM-L6-v2)")
print("This is a neural embedding model (similar to Word2Vec but sentence-level)")
print("=" * 80)

# Load pre-trained sentence embedding model
embedder = SentenceTransformer('all-MiniLM-L6-v2')
print("✓ Loaded embedding model")
print(f"✓ Embedding dimension: 384")

# Generate embeddings for all training samples
print("\n📊 Encoding training data...")
sentence_embeddings = embedder.encode(
    df['text'].tolist(), 
    convert_to_tensor=True,
    show_progress_bar=True
).to(device)

print(f"✓ Generated embeddings: {sentence_embeddings.shape}")
print(f"   - Shape: (num_samples, embedding_dim)")
print(f"   - Device: {sentence_embeddings.device}")



🧠 GENERATING SENTENCE EMBEDDINGS
Using: Sentence Transformers (all-MiniLM-L6-v2)
This is a neural embedding model (similar to Word2Vec but sentence-level)
✓ Loaded embedding model
✓ Embedding dimension: 384

📊 Encoding training data...


Batches: 100%|██████████| 21/21 [00:00<00:00, 60.92it/s]

✓ Generated embeddings: torch.Size([657, 384])
   - Shape: (num_samples, embedding_dim)
   - Device: cuda:0


In [7]:

# ============================================================================
# CELL 7: CNN MODEL ARCHITECTURE
# ============================================================================

print("\n" + "=" * 80)
print("🏗️  CNN MODEL ARCHITECTURE")
print("=" * 80)

class TextCNN(nn.Module):
    """
    Convolutional Neural Network for Text Classification
    
    Architecture:
    1. Input: Sentence embeddings (384-dim vectors)
    2. Multiple Conv1D layers with different kernel sizes (3, 4, 5)
       - Detects patterns of different n-gram lengths
    3. Max pooling: Extract most important features
    4. Dropout: Prevent overfitting (40%)
    5. Fully connected layer: Final classification
    
    Why CNN for text?
    - Detects local patterns (like phrases)
    - Translation invariant (same pattern anywhere in text)
    - Faster than RNN/LSTM
    - Simpler than Transformers
    """
    
    def __init__(self, embed_dim=384, num_classes=num_classes):
        super().__init__()
        
        # Multiple convolution layers with different kernel sizes
        # This captures n-grams of different lengths
        self.convs = nn.ModuleList([
            nn.Conv1d(in_channels=1, out_channels=128, kernel_size=k) 
            for k in [3, 4, 5]
        ])
        
        # Dropout for regularization
        self.dropout = nn.Dropout(0.4)
        
        # Fully connected output layer
        self.fc = nn.Linear(128 * 3, num_classes)
    
    def forward(self, x):
        # x shape: (batch_size, 384)
        x = x.unsqueeze(1)  # (batch_size, 1, 384)
        
        # Apply convolutions and max pooling
        convs = [F.relu(conv(x)).max(dim=2)[0] for conv in self.convs]
        
        # Concatenate all conv outputs
        x = torch.cat(convs, dim=1)  # (batch_size, 128*3)
        
        # Dropout and classification
        x = self.dropout(x)
        return self.fc(x)

# Print model architecture
print("\n📐 Model Architecture:")
print(TextCNN())
print("\n✓ Model defined successfully")


🏗️  CNN MODEL ARCHITECTURE

📐 Model Architecture:
TextCNN(
  (convs): ModuleList(
    (0): Conv1d(1, 128, kernel_size=(3,), stride=(1,))
    (1): Conv1d(1, 128, kernel_size=(4,), stride=(1,))
    (2): Conv1d(1, 128, kernel_size=(5,), stride=(1,))
  )
  (dropout): Dropout(p=0.4, inplace=False)
  (fc): Linear(in_features=384, out_features=33, bias=True)
)

✓ Model defined successfully


In [8]:

# ============================================================================
# CELL 8: TRAIN OR LOAD MODEL
# ============================================================================

print("\n" + "=" * 80)
print("🎯 TRAINING CNN MODEL")
print("=" * 80)

model_path = "cnn_chattea.pth"

try:
    # Try to load pre-trained model
    cnn_model = TextCNN().to(device)
    cnn_model.load_state_dict(torch.load(model_path, map_location=device))
    print("✓ Loaded pre-trained CNN model from", model_path)
    
except FileNotFoundError:
    print("⚠️  No pre-trained model found. Training from scratch...")
    print("\n" + "=" * 80)
    print("📚 PREPARING TRAINING DATA")
    print("=" * 80)
    
    # Encode all texts
    X = embedder.encode(df['text'].tolist(), convert_to_tensor=True).to(device)
    y = torch.tensor(df['label'].values, dtype=torch.long).to(device)
    
    print(f"✓ X shape: {X.shape}")
    print(f"✓ y shape: {y.shape}")
    
    # Train/validation split (stratified)
    train_idx, val_idx = train_test_split(
        torch.arange(len(X)),
        test_size=0.2,
        random_state=42,
        stratify=y.cpu()  # Stratify to maintain class distribution
    )
    
    X_train = X[train_idx].to(device)
    X_val = X[val_idx].to(device)
    y_train = y[train_idx].to(device)
    y_val = y[val_idx].to(device)
    
    print(f"\n✓ Training samples: {len(X_train)}")
    print(f"✓ Validation samples: {len(X_val)}")
    
    print("\n" + "=" * 80)
    print("🏋️  TRAINING LOOP")
    print("=" * 80)
    
    # Initialize model
    cnn_model = TextCNN().to(device)
    optimizer = torch.optim.Adam(cnn_model.parameters(), lr=0.002)
    criterion = nn.CrossEntropyLoss()
    
    # Training loop
    cnn_model.train()
    print("\nEpoch | Accuracy | Loss")
    print("-" * 40)
    
    for epoch in range(30):
        optimizer.zero_grad()
        
        # Forward pass
        outputs = cnn_model(X_train)
        loss = criterion(outputs, y_train)
        
        # Backward pass
        loss.backward()
        optimizer.step()
        
        # Calculate accuracy
        acc = (outputs.argmax(1) == y_train).float().mean().item()
        
        # Print progress every 5 epochs
        if epoch % 5 == 0:
            print(f"{epoch:5d} | {acc:8.4f} | {loss.item():8.4f}")
    
    # Final training accuracy
    with torch.no_grad():
        outputs = cnn_model(X_train)
        train_acc = (outputs.argmax(1) == y_train).float().mean().item()
        
        # Validation accuracy
        val_outputs = cnn_model(X_val)
        val_acc = (val_outputs.argmax(1) == y_val).float().mean().item()
    
    print("\n" + "=" * 80)
    print(f"✓ Final Training Accuracy: {train_acc:.4f} ({train_acc*100:.2f}%)")
    print(f"✓ Final Validation Accuracy: {val_acc:.4f} ({val_acc*100:.2f}%)")
    print("=" * 80)
    
    # Save model
    torch.save(cnn_model.state_dict(), model_path)
    print(f"\n✓ Model saved to: {model_path}")

# Set model to evaluation mode
cnn_model.eval()
print("\n✓ Model ready for inference!")



🎯 TRAINING CNN MODEL
⚠️  No pre-trained model found. Training from scratch...

📚 PREPARING TRAINING DATA
✓ X shape: torch.Size([657, 384])
✓ y shape: torch.Size([657])

✓ Training samples: 525
✓ Validation samples: 132

🏋️  TRAINING LOOP

Epoch | Accuracy | Loss
----------------------------------------
    0 |   0.0229 |   3.5025
    5 |   0.0305 |   3.5121
   10 |   0.0438 |   3.4987
   15 |   0.0362 |   3.4968
   20 |   0.0476 |   3.4842
   25 |   0.0324 |   3.4969

✓ Final Training Accuracy: 0.0324 (3.24%)
✓ Final Validation Accuracy: 0.0227 (2.27%)

✓ Model saved to: cnn_chattea.pth

✓ Model ready for inference!


In [9]:
# ============================================================================
# CELL 9: PHONE NUMBER EXTRACTION (NER)
# ============================================================================

print("\n" + "=" * 80)
print("📱 PHONE NUMBER EXTRACTION (Named Entity Recognition)")
print("=" * 80)

# Regex pattern for Indonesian phone numbers
PHONE_PATTERN = re.compile(
    r'(\b08[1-9]\d{7,12}\b|\b628[1-9]\d{7,12}\b|\b\+628[1-9]\d{7,12}\b)', 
    re.IGNORECASE
)

def extract_phone(text):
    """
    Extract and normalize Indonesian phone numbers
    
    Supports formats:
    - 08123456789 (local)
    - 628123456789 (international without +)
    - +628123456789 (international with +)
    
    Returns normalized format: 08xxxxxxxxxx
    """
    phone_match = PHONE_PATTERN.search(text)
    
    if phone_match:
        num = phone_match.group(0).replace(" ", "").replace("-", "")
        
        if num.startswith("08"):
            return num
        elif num.startswith("628"):
            return "0" + num[2:]
        elif num.startswith("+628"):
            return "0" + num[3:]
    
    return None

# Test phone extraction
print("\n🧪 Testing Phone Extraction:")
test_phones = [
    "check 08123456789",
    "verify 628123456789",
    "validate +628123456789",
    "is 0812-3456-789 valid"
]

for test in test_phones:
    phone = extract_phone(test)
    print(f"   '{test}' → {phone}")



📱 PHONE NUMBER EXTRACTION (Named Entity Recognition)

🧪 Testing Phone Extraction:
   'check 08123456789' → 08123456789
   'verify 628123456789' → 08123456789
   'validate +628123456789' → 08123456789
   'is 0812-3456-789 valid' → None


In [10]:
# ============================================================================
# CELL 10: MAIN CHAT FUNCTION (INFERENCE)
# ============================================================================

print("\n" + "=" * 80)
print("💬 CHAT INFERENCE FUNCTION")
print("=" * 80)

def get_chattea_reply(user_input: str) -> str:
    """
    Main chatbot inference function
    
    Pipeline:
    1. Rule-based filters (greetings, goodbyes)
    2. Phone number extraction (if applicable)
    3. CNN classification with confidence check
    4. Retrieval fallback (if low confidence)
    5. Response generation
    
    Args:
        user_input: User's message
        
    Returns:
        Bot's response
    """
    text = user_input.strip().lower()
    
    # ==================== RULE-BASED FILTERS ====================
    # Quick responses for common greetings
    if any(g in text for g in ["hai", "halo", "hello", "hi", "hey", "pagi", "siang", "malam"]):
        return RESPONSES["greeting"]["en"]
    
    if any(g in text for g in ["bye", "goodbye", "dadah", "sampai jumpa"]):
        return RESPONSES["goodbye"]["en"]
    
    # ==================== PHONE EXTRACTION ====================
    extracted_phone = extract_phone(user_input)
    
    # ==================== EMBEDDING + PREDICTION ====================
    with torch.no_grad():
        # Encode user input
        user_emb = embedder.encode(user_input, convert_to_tensor=True).to(device)
        user_emb = user_emb.unsqueeze(0)  # (1, 384)
        
        # CNN prediction
        cnn_logits = cnn_model(user_emb)
        cnn_probs = cnn_logits.softmax(1)
        cnn_confidence = cnn_probs.max().item()
        cnn_intent = intent_map[cnn_logits.argmax(1).item()]
        
        # Retrieval fallback (semantic similarity)
        cos_scores = util.cos_sim(user_emb, sentence_embeddings)[0]
        best_match_idx = cos_scores.argmax().item()
        retrieval_intent = df.iloc[best_match_idx]['intent']
        retrieval_score = cos_scores[best_match_idx].item()
        
        # Choose final intent based on confidence
        if cnn_confidence > 0.90:
            final_intent = cnn_intent
            source = "CNN"
        else:
            final_intent = retrieval_intent
            source = "Retrieval"
    
    # ==================== SPECIAL CASES ====================
    # Phone check with extracted number
    if final_intent == "phone_check" and extracted_phone:
        return f"Checking {extracted_phone}...\nYes, this number is registered and active on WhatsApp!\n\nYou can safely include it in your blast list."
    
    # ==================== RESPONSE GENERATION ====================
    response = RESPONSES.get(final_intent, RESPONSES.get("help", "I'm not sure how to help with that."))
    
    # Handle both dict (bilingual) and string responses
    if isinstance(response, dict):
        return response.get("en", response.get("id", "I'm not sure how to help with that."))
    
    return response

print("✓ Chat function ready!")


💬 CHAT INFERENCE FUNCTION
✓ Chat function ready!


In [11]:

# ============================================================================
# CELL 11: TEST INFERENCE
# ============================================================================

print("\n" + "=" * 80)
print("🧪 TESTING INFERENCE")
print("=" * 80)

test_queries = [
    "hello",
    "what is chattea",
    "how to blast message",
    "check 08123456789",
    "create instance",
    "schedule message",
    "thanks",
    "goodbye"
]

print("\nRunning test queries:\n")
for query in test_queries:
    print(f"👤 User: {query}")
    response = get_chattea_reply(query)
    print(f"🤖 Bot: {response[:100]}{'...' if len(response) > 100 else ''}")
    print("-" * 80)



🧪 TESTING INFERENCE

Running test queries:

👤 User: hello
🤖 Bot: Hello! 👋 Welcome to Chattea.

I'm here to help you navigate features like sending messages, managing...
--------------------------------------------------------------------------------
👤 User: what is chattea
🤖 Bot: Chattea is a WhatsApp marketing automation platform designed for businesses.

Key features:
• Send m...
--------------------------------------------------------------------------------
👤 User: how to blast message
🤖 Bot: To send a blast (mass message):

1. Go to **Blast Message** → **New Blast**
2. Select contacts, grou...
--------------------------------------------------------------------------------
👤 User: check 08123456789
🤖 Bot: Checking 08123456789...
Yes, this number is registered and active on WhatsApp!

You can safely inclu...
--------------------------------------------------------------------------------
👤 User: create instance
🤖 Bot: To create a new WhatsApp instance:

1. Open the **Instances** t

In [12]:

# ============================================================================
# CELL 13: MODEL EVALUATION
# ============================================================================

print("\n" + "=" * 80)
print("📊 MODEL EVALUATION")
print("=" * 80)

# Evaluate on validation set
with torch.no_grad():
    # Get embeddings
    X_all = embedder.encode(df['text'].tolist(), convert_to_tensor=True).to(device)
    y_all = torch.tensor(df['label'].values, dtype=torch.long).to(device)
    
    # Split
    train_idx, val_idx = train_test_split(
        torch.arange(len(X_all)),
        test_size=0.2,
        random_state=42,
        stratify=y_all.cpu()
    )
    
    X_val = X_all[val_idx].to(device)
    y_val = y_all[val_idx].to(device)
    
    # Predict
    val_outputs = cnn_model(X_val)
    val_preds = val_outputs.argmax(1)
    
    # Accuracy
    val_acc = (val_preds == y_val).float().mean().item()
    
    print(f"✓ Validation Accuracy: {val_acc:.4f} ({val_acc*100:.2f}%)")
    
    # Per-class accuracy
    print("\n📋 Per-Intent Performance:")
    for intent_id in range(num_classes):
        intent_name = intent_map[intent_id]
        mask = y_val == intent_id
        if mask.sum() > 0:
            intent_acc = (val_preds[mask] == y_val[mask]).float().mean().item()
            count = mask.sum().item()
            print(f"   {intent_name:30s}: {intent_acc:.3f} ({count:2d} samples)")

# Test on ALL training data (should be near perfect)
with torch.no_grad():
    all_outputs = cnn_model(sentence_embeddings.to(device))
    all_preds = all_outputs.argmax(1)
    all_labels = torch.tensor(df['label'].values, dtype=torch.long).to(device)
    
    train_acc = (all_preds == all_labels).float().mean().item()
    print(f"Accuracy on FULL training set: {train_acc:.4f}")

print("\n" + "=" * 80)
print("✅ NOTEBOOK COMPLETE!")
print("=" * 80)
print("\nYour model is ready to use!")

# ============================================================================
# FRESH START - DELETE EVERYTHING AND RETRAIN
# ============================================================================

import os

# 1. Delete saved model
if os.path.exists("cnn_chattea.pth"):
    os.remove("cnn_chattea.pth")
    print("✓ Deleted old model")

# 2. Clear GPU cache
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print("✓ Cleared GPU cache")

# 3. Restart notebook kernel (Kernel → Restart & Run All)
print("\n⚠️  NOW RESTART KERNEL AND RUN ALL CELLS FROM TOP!")


📊 MODEL EVALUATION
✓ Validation Accuracy: 0.0606 (6.06%)

📋 Per-Intent Performance:
   account                       : 0.000 ( 4 samples)
   analytics                     : 0.000 ( 4 samples)
   api                           : 0.000 ( 4 samples)
   cancel                        : 0.000 ( 4 samples)
   chat_view                     : 0.000 ( 4 samples)
   contacts                      : 0.000 ( 4 samples)
   definition                    : 0.750 ( 4 samples)
   files                         : 0.000 ( 4 samples)
   goodbye                       : 0.000 ( 4 samples)
   gratitude                     : 0.000 ( 4 samples)
   greeting                      : 0.750 ( 4 samples)
   groups                        : 0.000 ( 4 samples)
   help                          : 0.000 ( 4 samples)
   instance_connect              : 0.000 ( 4 samples)
   instance_create               : 0.000 ( 4 samples)
   instance_manage               : 0.000 ( 4 samples)
   instance_status               : 0.250 ( 4 sample